In [ ]:
import sys
!{sys.executable} -m pip install --user dit

In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam
from scipy.stats import entropy
from numpy.linalg import norm
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

In [3]:
def func(x, y):
    f1="../Proper Experiments/Parsed Datasets/"+x+".csv"
    f2="../Proper Experiments/Parsed Datasets/"+y+".csv"
    df = pd.read_csv(f1)
    df.text=df.text.astype(str)
    df_1 = pd.read_csv(f2)
    df_1.text=df_1.text.astype(str)
    corpus_1 = df['text'].values
    corpus_2 = df_1['text'].values
    text_1=""
    for i in corpus_1:
        text_1=text_1+i
    text_2=""
    for i in corpus_2:
        text_2=text_2+i    
    token = nltk.word_tokenize(text_1)
    unigrams_1 = ngrams(token,1)

    token = nltk.word_tokenize(text_2)
    unigrams_2 = ngrams(token,1)

    dict_1 = dict(Counter(unigrams_1))
    dict_2 = dict(Counter(unigrams_2))

    val=dict_1.values()
    temp=sum(val)
    for i in dict_1:
        dict_1[i]=dict_1[i]/temp

    val=dict_2.values()
    temp=sum(val)
    for i in dict_2:
        dict_2[i]=dict_2[i]/temp

    arr_1=[]
    arr_2=[]
    d1 = dict_1.copy()
    d2 = dict_2.copy()
    for i in dict_1:
        if i in dict_2:
            arr_1.append(d1[i])
            arr_2.append(d2[i])
            del d1[i]
            del d2[i]

    for i in d1:
        arr_1.append(d1[i])
        arr_2.append(0)

    for i in d2:
        arr_2.append(d2[i])
        arr_1.append(0)

    #print(len(arr_1)==len(arr_2))
    
    return (JSD(arr_1, arr_2))

In [55]:
print(JSD(arr_1, arr_2))

0.2708084591488323


In [4]:
file_names = ["Davidson","Waseem_Hovy", "OffensEval_2019", "hatEval_2019", "StormfrontWS", "GAB", "Reddit"]
ans = []
for i in file_names:
    temp = []
    for j in file_names:
        temp.append(func(i, j))
    ans.append(temp)
    print("Done")

Done
Done
Done
Done
Done
Done
Done


In [5]:
print(ans)

[[0.0, 0.292601498947455, 0.33216137946538016, 0.29260680130263295, 0.3598562841511459, 0.2928151050464475, 0.29206498798796654], [0.292601498947455, 0.0, 0.2834768806080229, 0.26407920790111505, 0.308104609777129, 0.23729516834112968, 0.21335683064375438], [0.33216137946538005, 0.2834768806080232, 0.0, 0.2708084591488324, 0.2937540220387137, 0.20304373392863598, 0.19542382409915726], [0.2926068013026331, 0.26407920790111516, 0.27080845914883245, 0.0, 0.3043615836162452, 0.21793842857281986, 0.22896732173684153], [0.3598562841511461, 0.30810460977712895, 0.2937540220387137, 0.3043615836162449, 0.0, 0.217633688807724, 0.2165194257196177], [0.29281510504644764, 0.2372951683411298, 0.20304373392863634, 0.2179384285728197, 0.217633688807724, 0.0, 0.11072327568462606], [0.2920649879879663, 0.21335683064375466, 0.19542382409915707, 0.22896732173684153, 0.21651942571961794, 0.11072327568462607, 0.0]]


In [7]:
dat=pd.DataFrame(ans)


In [8]:
dat.to_csv("Divergence Values.csv")